In [ ]:
import re


class UniversalPreprocessor:
    """
    Универсальный препроцессор текста:
    - нормализация пробелов
    - замена чисел -> <NUM>
    - аббревиатуры
    """

    def __init__(self,
                 replace_numbers: bool = True,
                 replace_urls: bool = True,
                 replace_emails: bool = True,
                 expand_abbreviations: bool = True):
        self.replace_numbers = replace_numbers
        self.replace_urls = replace_urls
        self.replace_emails = replace_emails
        self.expand_abbreviations = expand_abbreviations

        # сокращения
        self.abbreviations = {
            r"\bт\.е\.(?=\s|,|\.|$)": "то есть",
            r"\bт\.к\.(?=\s|,|\.|$)": "так как",
            r"\bи\.т\.д\.(?=\s|,|\.|$)": "и так далее"
        }

        # паттерны
        # даты вида 31.12.1999 или 31/12/1999
        self.date_pattern = re.compile(r"(?<!\d)(\d{1,2}[./]\d{1,2}[./]\d{2,4})(?!\d)")

        # числа
        self.num_pattern = re.compile(r"(?:(?<=^)|(?<=[^\w]))[+-]?\d+(?:[.,]\d+)?(?=$|[^\w])")

        self.url_pattern = re.compile(r"https?://\S+|www\.\S+")
        self.email_pattern = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b", flags=re.UNICODE)

    def preprocess(self, text: str) -> str:
        if not isinstance(text, str):
            return ""

        # нормализация юникод-пробелов + переводы строк ---
        text = text.replace("\u00A0", " ") 
        text = text.replace("\u202F", " ") 
        text = re.sub(r"[\r\t\n]+", " ", text)
        text = re.sub(r"\s+", " ", text).strip()

        # даты и числа ---
        if self.replace_numbers:
            # даты первыми, чтобы 31.12.1999 не распалось на числа
            text = self.date_pattern.sub("<DATE>", text)
            # числа
            text = self.num_pattern.sub("<NUM>", text)

        # г. → год (если после числа) ---
        text = re.sub(r"(?<=<NUM>)\s*г\.(?=\s|,|\.|$)", " год", text, flags=re.IGNORECASE)

        # г. → город (если перед заглавной буквой) ---
        text = re.sub(r"\bг\.(?=\s*[А-ЯЁ])", "город", text)

        # остальные сокращения ---
        if self.expand_abbreviations:
            for abbr, full in self.abbreviations.items():
                text = re.sub(abbr, full, text, flags=re.IGNORECASE)

        # URL и email ---
        if self.replace_urls:
            text = self.url_pattern.sub("<URL>", text)
        if self.replace_emails:
            text = self.email_pattern.sub("<EMAIL>", text)

        # финальная чистка пробелов ---
        text = re.sub(r"\s+", " ", text).strip()

        return text


In [80]:
if __name__ == "__main__":
    p = UniversalPreprocessor()
    s = (
        "1914. русские войска вступили пределы венгрии. "
        "Раннего утра 14 сентября 1914 г., г. Москва. "
        "Сайт: https://lenta.ru, email: test@example.com. Т.е. столица РФ."
    )
    print(p.preprocess(s))

<NUM>. русские войска вступили пределы венгрии. Раннего утра <NUM> сентября <NUM> год, город Москва. Сайт: <URL> email: <EMAIL>. то есть столица РФ.


In [ ]:
import json

input_file = "2_news_corpus_clean.jsonl"
output_file = "3_news_corpus_universal.jsonl"

preprocessor = UniversalPreprocessor()

count = 0
with open(input_file, "r", encoding="utf-8") as f_in, \
     open(output_file, "w", encoding="utf-8") as f_out:
    for line in f_in:
        if not line.strip():
            continue
        record = json.loads(line)

        title = record.get("title_clean") or record.get("title", "")
        text = record.get("text_clean") or record.get("text", "")

        # обработка
        record["title"] = preprocessor.preprocess(title)
        record["text"] = preprocessor.preprocess(text)

        f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
        count += 1

print(f"Обработано {count} записей. Результат сохранён в '{output_file}'.")


✅ Обработано 1000 записей. Результат сохранён в '3_news_corpus_universal.jsonl'.
